In [7]:
from cuml.datasets import make_regression
from cuml.preprocessing.model_selection import train_test_split
from cuml.linear_model import LinearRegression as cuLR
from sklearn.metrics import mean_absolute_error
import cupy as cp
from cupy import asnumpy
import time

In [8]:
n_samples = 2**10
n_features = 100
n_info = 70

X_reg, y_reg = make_regression( n_samples = n_samples, 
                                n_features = n_features,
                                n_informative = n_info, 
                                 )

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split( X_reg,
                                                                     y_reg, 
                                                                     train_size = 0.8,
                                                                      )
cuml_reg_model = cuLR( fit_intercept = False,
                       normalize = False,
                       algorithm = 'eig' )

trained_LR = cuml_reg_model.fit( X_reg_train, y_reg_train )
cu_preds = trained_LR.predict( X_reg_test )

In [9]:
def mean_absolute_err(y_true, y_pred,*, multioutput = 'raw_values'):
  output_errors = cp.average(cp.abs(y_pred - y_true), weights=None, axis=0)
  if isinstance(multioutput, str):
      if multioutput == 'raw_values':
          return output_errors
      elif multioutput == 'uniform_average':
            # pass None as weights to cp.average: uniform mean
          multioutput = None

  return cp.average(output_errors, weights=None)



In [10]:
y_reg_test = cp.asarray(y_reg_test) 
cu_preds = cp.asarray(cu_preds)

s = time.time()
cu_mae = mean_absolute_err( y_reg_test, cu_preds, multioutput='raw_values' )
e = time.time()
print("Time consumed by cuml: ", e-s)
p = time.time()
sk_mae = mean_absolute_error(asnumpy(y_reg_test),asnumpy(cu_preds), multioutput='raw_values')
d = time.time()
print("Time consumed by sklearn: ", d-p)


print("cuml's mae : ", cu_mae)
print("sklearn's mae : ", sk_mae)


Time consumed by cuml:  0.1917710304260254
Time consumed by sklearn:  0.0016658306121826172
cuml's mae :  [ 587.6687   575.3483   490.26956  442.1602   719.27167  431.5054
  606.88416  416.94858  427.81958  645.9032   547.3572   416.4214
  487.6155   712.32227  474.4517  1040.3994   457.03925  438.60013
  508.35443  779.8066   910.27905  615.2163   658.7811   491.07126
  736.7265   455.89597  416.92642  649.96234  423.4279   430.89474
  413.08408  830.8579   529.6624   490.81406  411.70062  530.203
  692.54144  421.8025   412.949    411.20804  714.7911   523.37286
  832.2852   412.13843 1077.8998   477.08902  412.5074   485.14856
  573.24945  523.2301   410.96494  615.4103   531.8932   846.075
  411.7443   445.6065   579.8226   430.7336   444.17728  589.0615
  584.5281   434.28595  461.7215   422.16855  411.01025  413.15967
  723.0263   423.99628  445.4663   518.1291   692.98047  412.577
  553.38763  991.3189   472.63055  457.83026  794.0624   434.41208
  543.9701   438.51645 1005.6523